In [64]:
import yfinance as yf
import numpy as np
import pandas as pd

from optimisation_markowitz import MarkowitzOptimization

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# get Date and Adj Close
def get_historical_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data = data[['Adj Close']]
    
    return data

We will use S&P500 Information Technology Index as benchmark to evaluate the performance with Single Index Market Model and Constant Correlation Model. The return over four years will be used as the market return in the formula

In [12]:
# Get close data for S&P 500 Information Technology Index
market_index_data = get_historical_data("^SP500-45", "2023-04-01", "2024-04-01")
market_index_data


[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2023-04-03,2637.780029
2023-04-04,2622.429932
2023-04-05,2591.169922
2023-04-06,2608.750000
2023-04-10,2604.909912
...,...
2024-03-22,3869.949951
2024-03-25,3849.919922
2024-03-26,3820.600098


For markowitz model, we will use 3-Month US Treasury Bill as risk-free rate. Since we are holding the portfolio for 1 year, we will take the last value as the risk-free rate.

In [84]:
risk_free_rate_df = yf.download("^IRX", start="2023-04-01", end="2024-04-01")
risk_free_rate = risk_free_rate_df['Adj Close'].iloc[-1] / 100
risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05203000068664551

We will now download the data for US Technology Equity ETFs on which we will run our strategies. We will use the following top 5 ETFs based on total assets:
1. Vanguard Information Technology ETF (VGT)
2. Technology Select Sector SPDR Fund (XLK)
3. VanEck Semiconductor ETF (SMH)
4. iShares US Technology ETF (IYW)
5. iShares Semiconductor ETF (SOXX)

Source: https://etfdb.com/etfdb-category/technology-equities/ 

In [20]:
# get the data for VanGuard Information Technology ETF
vgt_data = get_historical_data("VGT", "2023-04-01", "2024-04-01")
vgt_data

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2023-04-03,381.994537
2023-04-04,379.573334
2023-04-05,375.008820
2023-04-06,376.953674
2023-04-10,376.735352
...,...
2024-03-22,528.280029
2024-03-25,526.150024
2024-03-26,523.840027


In [21]:
# get the data for Technology Select Sector SPDR Fund
xlk_data = get_historical_data("XLK", "2023-04-01", "2024-04-01")
xlk_data

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2023-04-03,149.556702
2023-04-04,148.733383
2023-04-05,147.037140
2023-04-06,147.870392
2023-04-10,147.791031
...,...
2024-03-22,209.910004
2024-03-25,208.520004
2024-03-26,207.660004


In [22]:
# get the data for VanEck Semiconductor ETF
smh_data = get_historical_data("SMH", "2023-04-01", "2024-04-01")
smh_data

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2023-04-03,130.061035
2023-04-04,128.063095
2023-04-05,125.821594
2023-04-06,125.443871
2023-04-10,127.342430
...,...
2024-03-22,227.639999
2024-03-25,227.080002
2024-03-26,224.970001


In [28]:
# get the data for iShares US Technology ETF
iyt_data = get_historical_data("IYW", "2023-04-01", "2024-04-01")
iyt_data

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2023-04-03,92.403366
2023-04-04,91.975060
2023-04-05,90.829613
2023-04-06,91.646378
2023-04-10,91.506920
...,...
2024-03-22,136.589996
2024-03-25,135.899994
2024-03-26,135.210007


In [29]:
# get the data for iShares Semiconductor ETF
soxx_data = get_historical_data("SOXX", "2023-04-01", "2024-04-01")
soxx_data

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2023-04-03,143.788406
2023-04-04,141.107254
2023-04-05,138.660934
2023-04-06,137.946579
2023-04-10,140.383133
...,...
2024-03-22,225.449997
2024-03-25,224.570007
2024-03-26,222.880005


In [32]:
# check for any missing data
print(market_index_data.isnull().sum())
print(vgt_data.isnull().sum())
print(xlk_data.isnull().sum())
print(smh_data.isnull().sum())
print(iyt_data.isnull().sum())
print(soxx_data.isnull().sum())

Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64


In [66]:
# since, there is no missing data, we can go to the next step
# find simple return over the entire year
# Rt(k) = (Pt(k) - Pt(k-1)) / Pt(k-1)
# Rt(k) = (1 + Rt) * (1 + Rt-1) * ... * (1 + R1) - 1
def calculate_simple_return(data):
    data['simple_return'] = data['Adj Close'].pct_change()
    data['simple_return'] = data['simple_return'].fillna(0)
    data['simple_return'] = data['simple_return'] + 1
    data['simple_return'] = data['simple_return'].cumprod() - 1
    
    return data.iloc[-1]['simple_return']

In [85]:
market_index_data_return = calculate_simple_return(market_index_data)
vgt_data_return = calculate_simple_return(vgt_data)
xlk_data_return = calculate_simple_return(xlk_data)
smh_data_return = calculate_simple_return(smh_data)
iyt_data_return = calculate_simple_return(iyt_data)
soxx_data_return = calculate_simple_return(soxx_data)

equity_etf_returns = {
    "VGT": vgt_data_return,
    "XLK": xlk_data_return,
    "SMH": smh_data_return,
    "IYT": iyt_data_return,
    "SOXX": soxx_data_return
}
stock_returns_list = [vgt_data_return, xlk_data_return, smh_data_return, iyt_data_return, soxx_data_return]

In [86]:
# find the variance-covriance matrix of the returns
# create a dataframe with the returns of all the ETFs
# calculate the variance-covariance matrix
returns_df = pd.DataFrame([vgt_data['simple_return'], xlk_data['simple_return'], smh_data['simple_return'], iyt_data['simple_return'], soxx_data['simple_return']]).T
returns_df.columns = ["VGT", "XLK", "SMH", "IYT", "SOXX"]

cov_matrix = returns_df.cov()
cov_matrix

,VGT,XLK,SMH,IYT,SOXX
VGT,0.013222,0.014098,0.023890,0.015862,0.019918
XLK,0.014098,0.015095,0.025403,0.016997,0.021127
SMH,0.023890,0.025403,0.045854,0.028700,0.037523
IYT,0.015862,0.016997,0.028700,0.019211,0.023821
SOXX,0.019918,0.021127,0.037523,0.023821,0.031127


## Optimal tangency portfolio using Markowitz model 

In [87]:
# Run the Markowitz Portfolio Optimization
markowitz = MarkowitzOptimization(stock_returns_list, cov_matrix, risk_free_rate)


In [98]:
# find the weights of the portfolio with short shelling
weights_short_selling = markowitz.find_tangency_portfolio_with_short_selling()

In [100]:
# create the wieghts dictionary
weights_with_ss_dict_pct = {
    "VGT": weights_short_selling[0],
    "XLK": weights_short_selling[1],
    "SMH": weights_short_selling[2],
    "IYT": weights_short_selling[3],
    "SOXX": weights_short_selling[4]
}
weights_with_ss_dict_pct

{'VGT': -23.869302700943805,
 'XLK': 51.088280294463736,
 'SMH': -7.105883412559809,
 'IYT': -29.257905768751492,
 'SOXX': 10.144811587791368}

In [101]:
# find the weights of the portfolio without short shelling
weights_no_short_selling = markowitz.find_tangency_portfolio_no_short_selling()
weights_no_short_selling

array([9.14610696e-17, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00])

In [103]:
# weights dictionary
weights_no_ss_dict_pct = {
    "VGT": weights_no_short_selling[0],
    "XLK": weights_no_short_selling[1],
    "SMH": weights_no_short_selling[2],
    "IYT": weights_no_short_selling[3],
    "SOXX": weights_no_short_selling[4]
}
weights_no_short_selling

array([9.14610696e-17, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00])

In [104]:
# find the expected return and volatility of the portfolio with short selling
def calculate_portfolio_return_and_volatility(weights, returns):
    expected_return = np.dot(weights, returns)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    return expected_return, volatility

In [107]:
expected_return_with_ss, volatility_with_ss = calculate_portfolio_return_and_volatility(weights_short_selling, stock_returns_list)
print("Expected Return with Short Selling in %: ", expected_return_with_ss * 100)
print("Volatility with Short Selling in %: ", volatility_with_ss * 100)

Expected Return with Short Selling in %:  -173.64724444116658
Volatility with Short Selling in %:  38.88882525436394


In [108]:
expected_return_without_ss, volatility_without_ss = calculate_portfolio_return_and_volatility(weights_no_short_selling, stock_returns_list)
print("Expected Return without Short Selling in %: ", expected_return_without_ss * 100)
print("Volatility without Short Selling in %: ", volatility_without_ss * 100)

Expected Return without Short Selling in %:  72.98801691288277
Volatility without Short Selling in %:  21.413575710491575


In [121]:
# calculate the Sharpe Ratio
sharpe_ratio_without_ss = (expected_return_without_ss - risk_free_rate) / volatility_without_ss
sharpe_ratio_without_ss

3.165516014731111

From the results above, it seems like short selling in ETF stocks is not worth it. When not allowing short selling, the entire portfolio is invested in SMH. However, this is not a good strategy as it is not diversified. For portfolio with no short selling, we will add another constraint that the weight of each stock should be at max 30%. This will ensure that the portfolio is diversified.

In [117]:
weights_no_short_selling_with_constraint = markowitz.find_tangency_portfolio_no_short_selling_and_max_weight(0.3)
weights_no_short_selling_with_constraint

array([0.1, 0. , 0.3, 0.3, 0.3])

In [119]:
weights_dict_with_constraint = {
    "VGT": weights_no_short_selling_with_constraint[0],
    "XLK": weights_no_short_selling_with_constraint[1],
    "SMH": weights_no_short_selling_with_constraint[2],
    "IYT": weights_no_short_selling_with_constraint[3],
    "SOXX": weights_no_short_selling_with_constraint[4]
}
weights_dict_with_constraint

{'VGT': 0.09999999999999999,
 'XLK': 0.0,
 'SMH': 0.30000000000000027,
 'IYT': 0.3,
 'SOXX': 0.29999999999999993}

In [118]:
expected_return_with_ss_with_constraint, volatility_with_ss_with_constraint = calculate_portfolio_return_and_volatility(weights_no_short_selling_with_constraint, stock_returns_list)
print("Expected Return with Short Selling and Constraint in %: ", expected_return_with_ss_with_constraint * 100)
print("Volatility with Short Selling and Constraint in %: ", volatility_with_ss_with_constraint * 100)

Expected Return with Short Selling and Constraint in %:  56.607759060399744
Volatility with Short Selling and Constraint in %:  16.904883234084636


In [120]:
# find the Sharpe Ratio
sharpe_no_ss_with_constraint = (expected_return_with_ss_with_constraint - risk_free_rate) / volatility_with_ss_with_constraint
sharpe_no_ss_with_constraint

3.0408230734235326

After adding a constraint that each stock can have maximum of 30% weight, the portfolio is diversified. The portfolio is invested in VGT, SMH, IVT and SOXX with 10%, 30%, 30% and 30% weight respectively. The return on this portfolio is 56.61% with a risk of 16.90%. We see that the sharpe ratio by 0.12 to 3.04 when no short selling is allowed under weight constraint of 30%.